# Databricks Training: Import i eksploracja danych

**Moduł:** Fundamentals & Data Exploration  
**Temat:** DataFrame Reader API, opcje wczytywania, konstrukcja schematów, podstawowe operacje eksploracyjne  
**Czas trwania:** 20 minut  
**Poziom:** Początkujący

---

## Cel szkoleniowy

W tym notebooku nauczysz się:
- Używać DataFrame Reader API do wczytywania danych z różnych formatów
- Konfigurować opcje czytania danych (header, delimiter, inferSchema)
- Ręcznie definiować schematy za pomocą StructType i StructField
- Wykonywać podstawowe operacje eksploracyjne na DataFrame (columns, dtypes, count, describe, summary)
- Porównywać automatyczne wykrywanie schematów z ręczną definicją

---

## Kontekst biznesowy

Organizacja KION zarządza danymi z różnych źródeł:
- **Klienci** - dane w formacie CSV
- **Zamówienia** - dane w formacie JSON
- **Produkty** - dane w formacie Parquet

Zadaniem data engineera jest:
1. Zaimportować dane z różnych formatów do Databricks
2. Zrozumieć strukturę danych i typy kolumn
3. Przygotować dane do dalszych transformacji

---

## Podstawy teoretyczne

### DataFrame Reader API

Databricks udostępnia uniwersalne API do wczytywania danych:

```python
spark.read.format("format").option("key", "value").load("path")
```

Wspierane formaty:
- **CSV** - dane tabelaryczne z separatorami
- **JSON** - dane semi-strukturalne
- **Parquet** - kolumnowy format binarny z kompresją
- **Delta** - format transakcyjny (omówiony w kolejnych modułach)

### Opcje wczytywania danych

Dla **CSV**:
- `header=True` - pierwsza linia zawiera nazwy kolumn
- `inferSchema=True` - automatyczne wykrywanie typów danych
- `delimiter=","` - separator kolumn

Dla **JSON**:
- `multiLine=True` - JSON rozłożony na wiele linii
- `inferSchema=True` - wykrywanie struktury zagnieżdżonej

### Definicja schematu

Zalecana praktyka: **ręczne definiowanie schematu** zamiast automatycznego wykrywania.

Korzyści:
- Szybsze wczytywanie (brak konieczności skanowania danych)
- Kontrola typów danych
- Uniknięcie błędów przy niepoprawnych danych

Przykład:

```python
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("name", StringType(), True)
])
```

### Operacje eksploracyjne

| Operacja | Opis |
|----------|------|
| `df.columns` | Lista nazw kolumn |
| `df.dtypes` | Lista typów kolumn |
| `df.count()` | Liczba wierszy |
| `df.printSchema()` | Struktura schematu |
| `df.describe()` | Statystyki opisowe (count, mean, stddev, min, max) |
| `df.summary()` | Rozszerzone statystyki (+ percentyle) |
| `df.show(n)` | Wyświetl n wierszy |

---

## Inicjalizacja środowiska

Wykonujemy centralny skrypt konfiguracyjny:

In [0]:
%run ../00_setup

## Konfiguracja notebooka

Definiujemy zmienne specyficzne dla tego notebooka:

In [0]:
# Ścieżki do katalogów z danymi (podkatalogi w DATASET_BASE_PATH z 00_setup)
CUSTOMERS_PATH = f"{DATASET_BASE_PATH}/customers"
ORDERS_PATH = f"{DATASET_BASE_PATH}/orders"
PRODUCTS_PATH = f"{DATASET_BASE_PATH}/products"

# Ścieżki do konkretnych plików
CUSTOMERS_CSV = f"{CUSTOMERS_PATH}/customers.csv"
ORDERS_JSON = f"{ORDERS_PATH}/orders_batch.json"
PRODUCTS_PARQUET = f"{PRODUCTS_PATH}/products.parquet"

print(f"Ścieżka do pliku customers CSV: {CUSTOMERS_CSV}")
print(f"Ścieżka do pliku orders JSON: {ORDERS_JSON}")
print(f"Ścieżka do pliku products Parquet: {PRODUCTS_PARQUET}")

---

## Część 1: Import danych CSV (Customers)

### 1.1. Wczytanie CSV z automatycznym wykrywaniem schematu

In [0]:
# Wczytanie danych CSV z automatycznym wykrywaniem schematu
customers_auto_df = (
    spark.read
    .format("csv")
    .option("header", "true")       # Pierwsza linia to nazwy kolumn
    .option("inferSchema", "true")  # Automatyczne wykrywanie typów danych
    .load(CUSTOMERS_CSV)
)

In [0]:
# Wyświetl schemat
print(" Schemat wykryty automatycznie:")
customers_auto_df.printSchema()

In [0]:
print("\n Próbka danych (5 wierszy):")
customers_auto_df.show(5, truncate=False)

In [0]:
# Wyświetl próbkę danych
print("\n Próbka danych :")
display(customers_auto_df.head())

### 1.2. Eksploracja danych CSV

In [0]:
# Lista nazw kolumn
print(" Kolumny DataFrame:")
customers_auto_df.columns

In [0]:
# Lista typów danych
print("\n Typy danych kolumn:")
customers_auto_df.dtypes

In [0]:
# Liczba wierszy
row_count = customers_auto_df.count()
print(f"\nLiczba wierszy: {row_count}")

### 1.3. Statystyki opisowe

In [0]:
from pyspark.sql.types import StructType, StructField, StringType

# Definicja schematu dla customers
# Struktura rzeczywista: customer_id, first_name, last_name, email, phone, city, state, country, registration_date, customer_segment
customers_schema = StructType([
    StructField("customer_id", StringType(), nullable=False),
    StructField("first_name", StringType(), nullable=True),
    StructField("last_name", StringType(), nullable=True), 
    StructField("email", StringType(), nullable=True),
    StructField("phone", StringType(), nullable=True),
    StructField("city", StringType(), nullable=True),
    StructField("state", StringType(), nullable=True),
    StructField("country", StringType(), nullable=True),
    StructField("registration_date", StringType(), nullable=True),  # Date as string, will convert later
    StructField("customer_segment", StringType(), nullable=True)
])

# Statystyki opisowe (count, mean, stddev, min, max)
print(" Statystyki opisowe (describe):")
customers_auto_df.describe().display()

In [0]:
# Rozszerzone statystyki (+ percentyle)
print("\n Rozszerzone statystyki (summary):")
customers_auto_df.summary().display()

### 1.4. Ręczna definicja schematu dla CSV

**Best Practice:** Definiowanie schematu manualnie zapewnia kontrolę i wydajność.

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType

In [0]:
# Definicja schematu dla customers
# Struktura: customer_id (int), first_name (string), last_name (string), email (string), city (string), country (string), registration_date (timestamp)
customers_schema = StructType([
    StructField("customer_id", IntegerType(), nullable=False),
    StructField("first_name", StringType(), nullable=True),
    StructField("last_name", StringType(), nullable=True),
    StructField("city", StringType(), nullable=True),
    StructField("email", StringType(), nullable=True),
    StructField("country", StringType(), nullable=True),
    StructField("registration_date", TimestampType(), nullable=True)
])

In [0]:
# Wczytanie danych CSV z ręcznie zdefiniowanym schematem
customers_df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .schema(customers_schema)  # Użycie zdefiniowanego schematu
    .load(CUSTOMERS_CSV)
)
customers_df.printSchema()

In [0]:
print("\n Próbka danych z ręcznym schematem:")
customers_df.show(5, truncate=False)

### ✅ Ćwiczenie 1: Wczytanie CSV z własnymi opcjami

**Zadanie:** Wczytaj plik CSV `customers.csv` ponownie, ale:
- Ustaw delimiter na `;` (celowo błędny, aby zobaczyć efekt)
- Wyłącz `inferSchema`
- Zaobserwuj, jak to wpływa na wynik

**Hint:** Użyj `.option("delimiter", ";")`

In [0]:
# TODO: Wczytaj customers.csv z delimitorem `;` i bez inferSchema
# customers_wrong_df = spark.read.format("csv")...

# TODO: Wyświetl schemat i 5 wierszy
# customers_wrong_df.printSchema()
# customers_wrong_df.show(5, truncate=False)

# Zaobserwuj, że wszystkie dane będą w jednej kolumnie


---

## Część 2: Import danych JSON (Orders)

### 2.1. Wczytanie JSON z automatycznym wykrywaniem schematu

In [0]:
# Wczytanie danych JSON z automatycznym wykrywaniem schematu
orders_auto_df = (
    spark.read
    .format("json")
    .option("inferSchema", "true")
    .load(ORDERS_JSON)
)

print(" Schemat JSON wykryty automatycznie:")
orders_auto_df.printSchema()

In [0]:
print("\n Próbka danych JSON:")
orders_auto_df.display()

### 2.2. Eksploracja danych JSON

In [0]:
# Liczba kolumn i wierszy
print(f" Liczba kolumn: {len(orders_auto_df.columns)}")
print(f" Nazwy kolumn: {orders_auto_df.columns}")
print(f" Liczba wierszy: {orders_auto_df.count()}")

In [0]:
# Typy danych
print("\n Typy danych:")
for col_name, col_type in orders_auto_df.dtypes:
    print(f"  - {col_name}: {col_type}")

### 2.3. Ręczna definicja schematu dla JSON

In [0]:
from pyspark.sql.types import DoubleType, StringType, StructField, StructType, TimestampType, IntegerType

# Definicja schematu dla orders
# Struktura rzeczywista: order_id, customer_id, product_id, store_id, order_datetime, quantity, unit_price, discount_percent, total_amount, payment_method
orders_schema = StructType([
    StructField("order_id", StringType(), nullable=True),  # Może być null w danych
    StructField("customer_id", StringType(), nullable=False),
    StructField("product_id", StringType(), nullable=False),
    StructField("store_id", StringType(), nullable=False),
    StructField("order_datetime", StringType(), nullable=True),  # String, will convert to timestamp later
    StructField("quantity", IntegerType(), nullable=False),
    StructField("unit_price", DoubleType(), nullable=False),
    StructField("discount_percent", IntegerType(), nullable=False),
    StructField("total_amount", DoubleType(), nullable=False),
    StructField("payment_method", StringType(), nullable=False)
])

In [0]:
%skip
orders_schema = StructType([
    StructField("order_id", StringType(), nullable=False),
    StructField("customer_id", StringType(), nullable=True),
    StructField("order_datetime", TimestampType(), nullable=True),
    StructField("total_amount", DoubleType(), nullable=True),
    StructField("status", StringType(), nullable=True)
])

In [0]:
# Wczytanie danych JSON z ręcznie zdefiniowanym schematem
orders_df = (
    spark.read
    .format("json")
    .schema(orders_schema)
    .load(ORDERS_JSON)
)

print(" Schemat JSON zdefiniowany ręcznie:")
orders_df.printSchema()

print("\n Próbka danych z ręcznym schematem:")
orders_df.display(5, truncate=False)

### 2.4. Statystyki opisowe dla JSON

In [0]:
# Statystyki dla kolumn numerycznych
print(" Statystyki dla orders:")
orders_df.select("order_id", "customer_id", "total_amount").describe().display()

In [0]:
# Rozkład wartości dla kolumny 'status'
print("\n Rozkład wartości w kolumnie 'status':")
orders_df.groupBy("customer_id").count().orderBy("count", ascending=False).display()

In [0]:
# Rozkład wartości dla kolumny 'status'
print("\n Rozkład wartości w kolumnie 'status':")
orders_auto_df.groupBy("customer_id").count().orderBy("count", ascending=False).display()

### ✅ Ćwiczenie 2: Eksploracja danych JSON

**Zadanie:**
1. Policz liczbę unikalnych klientów w DataFrame `orders_auto_df`
2. Znajdź wartość maksymalną i minimalną w kolumnie `total_amount`
3. Policz liczbę zamówień w każdym statusie

**Hint:** Użyj `.select()`, `.distinct()`, `.count()`, `.agg()`, `.groupBy()`

In [0]:
# TODO: Policz liczbę unikalnych customer_id
# unique_customers = orders_auto_df.select("customer_id").____().count()
# print(f"Liczba unikalnych klientów: {unique_customers}")

# TODO: Znajdź min i max total_amount
# from pyspark.sql.functions import min, max
# orders_auto_df.select(min("____"), __("total_amount")).display()

# TODO: Policz zamówienia według statusu
# orders_auto_df.groupBy("____").count().display()

pass

---

## Część 3: Import danych Parquet (Products)

### 3.1. Wczytanie Parquet (schemat wbudowany)

In [0]:
# Parquet zawiera już schemat wbudowany - nie trzeba go definiować
products_df = (
    spark.read
    .format("parquet")
    .load(PRODUCTS_PARQUET)
)

print(" Schemat Parquet (wbudowany):")
products_df.printSchema()

print("\n Próbka danych Parquet:")
products_df.display(5, truncate=False)

### 3.2. Eksploracja danych Parquet

In [0]:
# Podstawowe informacje
print(f" Liczba kolumn: {len(products_df.columns)}")
print(f" Nazwy kolumn: {products_df.columns}")
print(f" Liczba produktów: {products_df.count()}")


In [0]:
# Typy danych
print("\n Typy danych:")
for col_name, col_type in products_df.dtypes:
    print(f"  - {col_name}: {col_type}")

### 3.3. Statystyki opisowe dla Parquet

In [0]:
# Statystyki dla kolumn numerycznych
print(" Statystyki dla products:")
products_df.describe().display()


In [0]:
# Rozszerzone statystyki
print("\n Rozszerzone statystyki:")
products_df.summary().display()

### ✅ Ćwiczenie 3: Analiza danych Parquet

**Zadanie:**
1. Znajdź średnią cenę produktu (jeśli kolumna `price` istnieje)
2. Znajdź 5 najdroższych produktów
3. Policz produkty według kategorii (jeśli kolumna istnieje)

**Hint:** Użyj `.agg()`, `.orderBy()`, `.limit()`

In [0]:
from pyspark.sql.functions import avg
# TODO: Oblicz średnią cenę
products_df.select(avg("_____")).display()

# TODO: Znajdź 5 najdroższych produktów
products_df.orderBy(col("_____").desc()).limit(5).display()

# TODO: Policz produkty według kategorii
if "category" in products_df.columns:
    products_df.groupBy("category").count().display()


---

## Część 4: Porównanie wydajności

### 4.1. Wczytanie CSV: inferSchema vs ręczny schemat

In [0]:
import time

# Test 1: Automatyczne wykrywanie schematu
start_auto = time.time()
df_auto = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(CUSTOMERS_CSV)
)
count_auto = df_auto.count()  # Action - wymusza wykonanie
time_auto = time.time() - start_auto

print(f"[BENCHMARK] Wczytanie CSV z inferSchema: {time_auto:.3f} sekund")
print(f"[BENCHMARK] Liczba wierszy: {count_auto}")

# Test 2: Ręczne zdefiniowanie schematu
start_manual = time.time()
df_manual = (
    spark.read
    .format("csv")
    .option("header", "true")
    .schema(customers_schema)
    .load(CUSTOMERS_CSV)
)
count_manual = df_manual.count()  # Action - wymusza wykonanie
time_manual = time.time() - start_manual

print(f"\n[BENCHMARK] Wczytanie CSV z ręcznym schematem: {time_manual:.3f} sekund")
print(f"[BENCHMARK] Liczba wierszy: {count_manual}")

# Porównanie
speedup = (time_auto - time_manual) / time_auto * 100
print(f"\n[WYNIK] Przyspieszenie: {speedup:.1f}%")

---

## Część 5: Best Practices

### Zalecenia przy importowaniu danych:

1. **Zawsze definiuj schemat ręcznie**
   - Szybsze wczytywanie
   - Kontrola typów danych
   - Uniknięcie błędów typu

2. **Wybieraj odpowiedni format**
   - **Parquet** - najlepszy dla analityki (kolumnowy, kompresja)
   - **CSV** - łatwy do debugowania, ale wolniejszy
   - **JSON** - elastyczny dla danych semi-strukturalnych

3. **Używaj partycjonowania**
   - Przyspieszenie zapytań filtrujących
   - Przykład: partycjonowanie zamówień według daty

4. **Sprawdzaj jakość danych od razu**
   - `count()` - sprawdź liczbę wierszy
   - `describe()` - sprawdź rozkład wartości
   - `printSchema()` - zweryfikuj typy

5. **Używaj `limit()` podczas eksperymentowania**
   - Przyspiesza iteracje nad kodem
   - Przykład: `df.limit(1000).display()`

6. **Dokumentuj schematy**
   - Ułatwia utrzymanie kodu
   - Przykład: komentarze przy definicji StructType

---

## Podsumowanie

W tym notebooku nauczyłeś się:

✅ **DataFrame Reader API**
- Wczytywanie danych z CSV, JSON, Parquet
- Konfiguracja opcji (header, inferSchema, delimiter)

✅ **Ręczna definicja schematu**
- Tworzenie schematów za pomocą StructType i StructField
- Porównanie wydajności: inferSchema vs ręczny schemat

✅ **Operacje eksploracyjne**
- Podstawowe operacje: columns, dtypes, count
- Statystyki: describe(), summary()
- Grupowanie i agregacja

✅ **Best Practices**
- Zalecenie ręcznego definiowania schematu
- Wybór formatu danych dla różnych scenariuszy
- Sprawdzanie jakości danych

---

### Następne kroki:

📌 **Kolejny notebook:** `03_basic_transformations_sql_pyspark.ipynb`  
📌 **Temat:** Transformacje danych, operacje SQL i PySpark API

---

## Zasoby dodatkowe

- [Databricks - Reading Data](https://docs.databricks.com/ingestion/index.html)
- [PySpark DataFrame API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html)

---